# Techno-economic analysis

[TEA](../TEA.txt) objects can perform cashflow analysis on a [System](../System.txt) object and arguments for cashflow analysis. These include arguments such as operating days, lang factor, and income tax, as well as arguments for taking into account startup operation, construction schedule, and capital cost financing.

### Inhereting from TEA objects

Depending on the rigour and flexibility of the techno-economic analysis, different parameters may be needed to calculate total depreciable capital (TDC), fixed capital investment (FCI), and fixed operating cost (FOC). For this reason, the TEA object is left as an *abstract class* with *abstract methods* `_TDC`, `_FCI`, and `_FOC`. Here is an example TEA subclass from the lipidcane biorefinery design for the co-production of ethanol and biodiesel:

In [1]:
import biosteam as bst

class LipidcaneTEA(bst.TEA):
    """
    Create a LipidcaneTEA object for techno-economic analysis of a biorefinery [1]_.
    
    Parameters
    ----------    
    system : System
        Should contain feed and product streams.
    IRR : float
        Internal rate of return (fraction).
    duration : tuple[int, int]
        Start and end year of venture (e.g. (2018, 2038)).
    depreciation : str
        'MACRS' + number of years (e.g. 'MACRS7').
    operating_days : float
        Number of operating days per year.
    income_tax : float
        Combined federal and state income tax rate (fraction).
    lang_factor : float
        Lang factor for getting fixed capital investment from
        total purchase cost. If no lang factor, estimate capital investment
        using bare module factors.
    startup_schedule : tuple[float]
        Startup investment fractions per year 
        (e.g. (0.5, 0.5) for 50% capital investment in the first year and 50%
        investment in the second).
    WC_over_FCI : float
        Working capital as a fraction of fixed capital investment.
    labor_cost : float
        Total labor cost (USD/yr).
    fringe_benefits : float
        Cost of fringe benefits as a fraction of labor cost.
    property_tax : float
        Fee as a fraction of fixed capital investment.
    property_insurance : float
        Fee as a fraction of fixed capital investment.    
    supplies : float
        Yearly fee as a fraction of labor cost.
    maintenance : float
        Yearly fee as a fraction of fixed capital investment.
    administration : float
        Yearly fee as a fraction of fixed capital investment.

    References
    ----------
    .. [1] Huang, H., Long, S., & Singh, V. (2016). Techno-economic analysis of biodiesel
        and ethanol co-production from lipid-producing sugarcane. Biofuels, Bioproducts
        and Biorefining, 10(3), 299–315. https://doi.org/10.1002/bbb.1640
    
    """
    
    def __init__(self, system, IRR, duration, depreciation, income_tax,
                 operating_days, lang_factor, construction_schedule, WC_over_FCI,
                 labor_cost, fringe_benefits, property_tax,
                 property_insurance, supplies, maintenance, administration):
        # Huang et. al. does not take into account financing or startup
        # so these parameters are 0 by default
        super().__init__(system, IRR, duration, depreciation, income_tax,
                         operating_days, lang_factor, construction_schedule,
                         startup_months=0, startup_FOCfrac=0, startup_VOCfrac=0,
                         startup_salesfrac=0, finance_interest=0, finance_years=0, 
                         finance_fraction=0, WC_over_FCI=WC_over_FCI)
        self.labor_cost = labor_cost
        self.fringe_benefits = fringe_benefits
        self.property_tax = property_tax
        self.property_insurance = property_insurance
        self.supplies= supplies
        self.maintenance = maintenance
        self.administration = administration
        
    # The abstract _DPI method should take installed equipment cost
    # and return the direct permanent investment. Huang et. al. assume 
    # these values are equal
    def _DPI(self, installed_equipment_cost):
        return installed_equipment_cost
    
    # The abstract _TDC method should take direct permanent investment
    # and return the total depreciable capital. Huang et. al. assume 
    # these values are equal
    def _TDC(self, DPI):
        return DPI
    
    # The abstract _FCI method should take total depreciable capital
    # and return the fixed capital investment. Again, Huang et. al.  
    # assume these values are equal.
    def _FCI(self, TDC):
        return TDC
    
    # The abstract _FOC method should take fixed capital investment
    # and return the fixed operating cost.
    def _FOC(self, FCI):
        return (FCI*(self.property_tax + self.property_insurance
                     + self.maintenance + self.administration)
                + self.labor_cost*(1+self.fringe_benefits+self.supplies))

### Perform cash flow analysis

Create a TEA object from a system:

In [2]:
from biorefineries import lipidcane as lc

lipidcane_tea = LipidcaneTEA(system=lc.lipidcane_sys,
                             IRR=0.15,
                             duration=(2018, 2038),
                             depreciation='MACRS7',
                             income_tax=0.35,
                             operating_days=200,
                             lang_factor=3,
                             construction_schedule=(0.4, 0.6),
                             WC_over_FCI=0.05,
                             labor_cost=2.5e6,
                             fringe_benefits=0.4,
                             property_tax=0.001,
                             property_insurance=0.005,
                             supplies=0.20,
                             maintenance=0.01,
                             administration=0.005)

lipidcane_tea.show() # Print TEA summary at current options
# Ignore the warnings, these are taken care of internally.

LipidcaneTEA: lipidcane_sys
 NPV: 38,722,133 USD at 15.0% IRR


C:\Users\yrc2\OneDrive\Code\biosteam\biosteam\units\design_tools\pressure_vessel.py:85: UserWarning: vacuum pressure vessel ASME codes not implemented yet; wall thickness may be inaccurate and stiffening rings may be required
  warn('vacuum pressure vessel ASME codes not implemented yet; '


Retrieve complete cashflow analysis as a DataFrame object:

In [3]:
lipidcane_tea.get_cashflow_table()

Depreciable capital  Fixed capital investment  Working capital  \
2016             8.87e+07                  8.87e+07                0   
2017             1.33e+08                  1.33e+08         1.11e+07   
2018                    0                         0                0   
2019                    0                         0                0   
2020                    0                         0                0   
2021                    0                         0                0   
2022                    0                         0                0   
2023                    0                         0                0   
2024                    0                         0                0   
2025                    0                         0                0   
2026                    0                         0                0   
2027                    0                         0                0   
2028                    0                         0                0   
2029                    0                         0                0   
2030                    0                         0                0   
2031                    0                         0                0   
2032                    0                         0                0   
2033                    0                         0                0   
2034                    0                         0                0   
2035                    0                         0                0   
2036                    0                         0                0   
2037                    0                         0        -1.11e+07   

      Depreciation  Loan  ...  Net earnings  Cash flow  Discount factor  \
2016             0     0  ...             0  -8.87e+07             1.15   
2017             0     0  ...             0  -1.44e+08                1   
2018      3.17e+07     0  ...      3.54e+07    6.7e+07             0.87   
2019      5.43e+07     0  ...      4.84e+06   5.91e+07            0.756   
2020      3.88e+07     0  ...      2.58e+07   6.46e+07            0.658   
2021      2.77e+07     0  ...      4.08e+07   6.84e+07            0.572   
2022      1.98e+07     0  ...      5.14e+07   7.12e+07            0.497   
2023      1.98e+07     0  ...      5.14e+07   7.12e+07            0.432   
2024      1.98e+07     0  ...      5.14e+07   7.12e+07            0.376   
2025      9.89e+06     0  ...      6.48e+07   7.47e+07            0.327   
2026             0     0  ...      7.81e+07   7.81e+07            0.284   
2027             0     0  ...      7.81e+07   7.81e+07            0.247   
2028             0     0  ...      7.81e+07   7.81e+07            0.215   
2029             0     0  ...      7.81e+07   7.81e+07            0.187   
2030             0     0  ...      7.81e+07   7.81e+07            0.163   
2031             0     0  ...      7.81e+07   7.81e+07            0.141   
2032             0     0  ...      7.81e+07   7.81e+07            0.123   
2033             0     0  ...      7.81e+07   7.81e+07            0.107   
2034             0     0  ...      7.81e+07   7.81e+07           0.0929   
2035             0     0  ...      7.81e+07   7.81e+07           0.0808   
2036             0     0  ...      7.81e+07   7.81e+07           0.0703   
2037             0     0  ...      7.81e+07   8.92e+07           0.0611   

      Net present value (NPV)  Cumulative NPV  
2016                -1.02e+08       -1.02e+08  
2017                -1.44e+08       -2.46e+08  
2018                 5.83e+07       -1.88e+08  
2019                 4.47e+07       -1.43e+08  
2020                 4.25e+07       -1.01e+08  
2021                 3.91e+07       -6.15e+07  
2022                 3.54e+07       -2.61e+07  
2023                 3.08e+07         4.7e+06  
2024                 2.68e+07        3.15e+07  
2025                 2.44e+07        5.59e+07  
2026                 2.22e+07        7.81e+07  
2027                 1.93e+07        9.74e+07  

### Find production cost, price, and IRR

Find production cost:

In [4]:
products = [bst.main_flowsheet('ethanol'), bst.main_flowsheet('biodiesel')]
lipidcane_tea.production_cost(products)# USD/yr

array([24252728.857, 31957569.678])

Solve for the price of a stream at the break even point:

In [5]:
feed = bst.main_flowsheet('lipidcane')
lipidcane_tea.solve_price(feed) # USD/kg

0.049451614883489196

Solve for the IRR at the break even point:

In [6]:
lipidcane_tea.IRR = lipidcane_tea.solve_IRR()
lipidcane_tea.show()

LipidcaneTEA: lipidcane_sys
 NPV: 2 USD at 17.8% IRR


Note that the cashflow table is created up to date:

In [7]:
lipidcane_tea.get_cashflow_table()

Depreciable capital  Fixed capital investment  Working capital  \
2016             8.87e+07                  8.87e+07                0   
2017             1.33e+08                  1.33e+08         1.11e+07   
2018                    0                         0                0   
2019                    0                         0                0   
2020                    0                         0                0   
2021                    0                         0                0   
2022                    0                         0                0   
2023                    0                         0                0   
2024                    0                         0                0   
2025                    0                         0                0   
2026                    0                         0                0   
2027                    0                         0                0   
2028                    0                         0                0   
2029                    0                         0                0   
2030                    0                         0                0   
2031                    0                         0                0   
2032                    0                         0                0   
2033                    0                         0                0   
2034                    0                         0                0   
2035                    0                         0                0   
2036                    0                         0                0   
2037                    0                         0        -1.11e+07   

      Depreciation  Loan  ...  Net earnings  Cash flow  Discount factor  \
2016             0     0  ...             0  -8.87e+07             1.18   
2017             0     0  ...             0  -1.44e+08                1   
2018      3.17e+07     0  ...      3.54e+07    6.7e+07            0.849   
2019      5.43e+07     0  ...      4.84e+06   5.91e+07             0.72   
2020      3.88e+07     0  ...      2.58e+07   6.46e+07            0.611   
2021      2.77e+07     0  ...      4.08e+07   6.84e+07            0.519   
2022      1.98e+07     0  ...      5.14e+07   7.12e+07             0.44   
2023      1.98e+07     0  ...      5.14e+07   7.12e+07            0.374   
2024      1.98e+07     0  ...      5.14e+07   7.12e+07            0.317   
2025      9.89e+06     0  ...      6.48e+07   7.47e+07            0.269   
2026             0     0  ...      7.81e+07   7.81e+07            0.228   
2027             0     0  ...      7.81e+07   7.81e+07            0.194   
2028             0     0  ...      7.81e+07   7.81e+07            0.164   
2029             0     0  ...      7.81e+07   7.81e+07             0.14   
2030             0     0  ...      7.81e+07   7.81e+07            0.118   
2031             0     0  ...      7.81e+07   7.81e+07              0.1   
2032             0     0  ...      7.81e+07   7.81e+07           0.0853   
2033             0     0  ...      7.81e+07   7.81e+07           0.0724   
2034             0     0  ...      7.81e+07   7.81e+07           0.0614   
2035             0     0  ...      7.81e+07   7.81e+07           0.0521   
2036             0     0  ...      7.81e+07   7.81e+07           0.0442   
2037             0     0  ...      7.81e+07   8.92e+07           0.0375   

      Net present value (NPV)  Cumulative NPV  
2016                -1.04e+08       -1.04e+08  
2017                -1.44e+08       -2.49e+08  
2018                 5.69e+07       -1.92e+08  
2019                 4.26e+07       -1.49e+08  
2020                 3.95e+07        -1.1e+08  
2021                 3.55e+07       -7.42e+07  
2022                 3.13e+07       -4.28e+07  
2023                 2.66e+07       -1.62e+07  
2024                 2.26e+07        6.36e+06  
2025                 2.01e+07        2.64e+07  
2026                 1.78e+07        4.43e+07  
2027                 1.51e+07        5.94e+07  

### System report

Save stream tables, utility requirements, itemized costs, TEA results, and a cash flow table:

In [8]:
# Try this on your computer and open excel
lc.lipidcane_sys.save_report('Lipidcane report.xlsx')
# Ignore the warning. The flowsheet is saved on the excel file
# as a really big image and Python thinks it could be a 
# malicious file cause its so big.